# Binary Search

**Thought:**

The core idea here is to make the search space become smaller dramatically, the normal way is to make the search space become only halp of the original one.

The prunning depends on the rule governning the array, like sorted.

### Search space reduction

A good problem for search in a sorted matrix, with shows how to reduce the search space wisely. [link](https://leetcode.com/articles/search-a-2d-matrix-ii/)

In [1]:
class Solution:
    def searchMatrix(self, matrix, target):
        # an empty matrix obviously does not contain `target` (make this check
        # because we want to cache `width` for efficiency's sake)
        if len(matrix) == 0 or len(matrix[0]) == 0:
            return False

        # cache these, as they won't change.
        height = len(matrix)
        width = len(matrix[0])

        # start our "pointer" in the bottom-left
        row = height-1
        col = 0

        while col < width and row >= 0:
            if matrix[row][col] > target:
                row -= 1
            elif matrix[row][col] < target:
                col += 1
            else: # found it
                return True
        
        return False

### Make use of binary tree features

For complete binary tree, here is a [problem](https://leetcode.com/problems/count-complete-tree-nodes/description/)

In [2]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def countNodes(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        if not root:
            return 0
        ans = 0
        layer = 0
        def helper(root):
            layer = 0
            while root:
                layer += 1
                root = root.right
            return layer
        ans = 0
        while root:
            ans += 1
            if not root.left and not root.right:
                return ans
            if root.left and not root.right:
                return ans + 1
            leftlayer = helper(root.left)
            rightlayer = helper(root.right)
            if leftlayer == rightlayer:
                ans += 2**(rightlayer)-1
                root = root.left
            elif leftlayer > rightlayer:
                ans += 2**(leftlayer)-1
                root = root.right

### Use binary search to enhance dynamic programming

Problem [link](https://leetcode.com/problems/longest-increasing-subsequence/discuss/74824/JavaPython-Binary-search-O(nlogn)-time-with-explanation)

In [8]:
class Solution(object):
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n = len(nums)
        # dp[i]: for a sequence with length i+1, the last number will be dp[i]
        dp = [0]*n
        size = 0
        for num in nums:
            left, right = 0, size
            while left < right:
                mid = (left+right)//2
                if dp[mid] < num:
                    left = mid+1
                else:
                    right = mid
            dp[right] = num
            size = max(size, right+1)
        return size